In [17]:
import os
import json
from bs4 import BeautifulSoup


In [8]:
# Assuming the JSON file contains a list of documents
file_path = '../data/document.json'
output_directory = '../data/documents'

In [5]:
os.makedirs(output_directory, exist_ok=True)


In [22]:
with open(file_path, 'r') as file:
    documents = file.read()
    
for doc in documents.splitlines():
    data = json.loads(doc)
    pid = data['prolific_id']
    content = data['content']
    with open(f'../data/documents/{pid}.html', 'w') as file:
        file.write(content)

In [18]:
tmp = json.loads(documents.splitlines()[0])['content']

In [21]:
with open('../data/doc1.html', 'w') as file:
        file.write(tmp)

In [19]:

# Extracting the content
html_content = tmp

# Converting HTML content to plain text using BeautifulSoup
soup = BeautifulSoup(html_content, "html.parser")
plain_text = soup.get_text()

# Creating a PDF document

In [20]:
plain_text

'Feel free to play around with the features before you begin your new task.\nHere are some steps you may want to go through:\nBelow this type @ai followed by some text to ask AI to write a piece of text\nOnce AI generated the paragraph select the paragraph, right-click, and select Create variation.\nClick different modification one by one and observe how your original text is modified\nFinally type an incomplete sentence and right-click, and select Create continuation. See how AI will complete your sentence\nNote: Instructions can be found in the top left corner.\nNota: Haga clic en "Save and Continue" una vez que haya explorado las funciones de la herramienta.'